# End of week 1 exercise

To demonstrate your familiarity with OpenAI API, and also Ollama, build a tool that takes a technical question,  
and responds with an explanation. This is a tool that you will be able to use yourself during the course!

In [1]:
# imports

import ollama
import os
from dotenv import load_dotenv
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [2]:
# constants

MODEL_GPT = 'gpt-4o-mini'
MODEL_LLAMA = 'llama3.2'

In [3]:
# set up environment

# Retrieve the 'OPENAI_API_KEY' value from the environment variables
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")

# Check if the API key is valid by looking for specific patterns
if api_key and api_key.startswith("sk-proj-") and len(api_key) > 10:
    print("API key looks good so far")
else:
    print(
        "There might be a problem with your API key? Please visit the troubleshooting notebook!"
    )

# Create an instance of OpenAI to interact with the OpenAI API
openai = OpenAI()

API key looks good so far


In [4]:
# here is the question; type over this to ask something new

question = input("Please, enter your question here:\n")

# Example question:
# Please explain what this code does and why: yield from {book.get("author") for book in books if book.get("author")}

Please, enter your question here:
 Please explain what this code does and why: yield from {book.get("author") for book in books if book.get("author")}


In [5]:
tutor_system_prompt = """Role and Purpose: You are a tutor for the "Large Language Model Engineering" 
course by Ed Donner. Your role is to provide structured, clear, and patient explanations, helping 
learners understand concepts and apply them effectively.


Start with context, progressing from basic overviews to detailed insights.
Use concise language, real-world examples, and analogies to simplify complex ideas.
Summarize key points, invite further questions and suggest further resources for exploration.


Tone:
Maintain professionalism, avoid assumptions about learners' backgrounds, and focus strictly
on course-relevant content. Use examples and citations aligned with the course material.
"""

user_prompt = "Please give a detailed explanation to the following question: " + question

In [6]:
tutor_messages = [
        {"role": "system", "content": tutor_system_prompt},
        {"role": "user", "content": user_prompt}
]

In [7]:
# Get gpt-4o-mini to answer, with streaming

def display_explanation():
    stream = openai.chat.completions.create(
        model=MODEL_GPT,
        messages=tutor_messages,
        stream=True,
    )

    explanation = ""
    display_handle = display(Markdown(""), display_id=True)

    for chunk in stream:
       
        explanation += chunk.choices[0].delta.content or ""
        explanation = explanation.replace("```", "").replace("markdown", "")
        update_display(Markdown(explanation), display_id=display_handle.display_id)

In [8]:
display_explanation()

Certainly! Let's break down the code snippet you've provided: 

python
yield from {book.get("author") for book in books if book.get("author")}


### Context and Overview

In Python, the `yield from` statement is used within a generator function to yield all values from an iterable (like a list, set, or a generator) one by one. The expression `{book.get("author") for book in books if book.get("author")}` is a set comprehension that creates a set of authors from the list of books, `books`, filtering out any entries that do not have an author.

### Detailed Breakdown

1. **`{book.get("author") for book in books if book.get("author")}`**:
   - This is a **set comprehension**, a concise way to create a set.
   - The expression iterates over each `book` in the iterable `books`.
   - `book.get("author")` attempts to retrieve the value associated with the key `"author"` from `book`.
   - The `if book.get("author")` condition filters out the books where the author is `None` or does not exist (i.e., books without an author).
   - The result is a set containing all unique author names from the `books` list.

2. **`yield from`**:
   - The `yield from` expression is a generator feature that allows you to delegate part of the generator’s operations to another generator. It simplifies yielding multiple values.
   - Thus, `yield from` will yield each author from the set of authors generated above.

### Why This Code Is Useful

- **Eliminating Duplicates**: Using a set comprehension ensures that even if multiple books are authored by the same person, that author's name will only appear once in the final output.
- **Memory Efficiency**: Since sets are constructed in memory, it deals efficiently with the uniqueness and further processing of authors.
- **Clean and Concise**: This one-liner allows you to gather authors succinctly and effectively without needing temporary lists or verbose loops.

### Example

Imagine you have the following list of books:

python
books = [
    {"title": "Book One", "author": "Author A"},
    {"title": "Book Two", "author": "Author B"},
    {"title": "Book Three", "author": "Author A"},
    {"title": "Book Four"},
    {"title": "Book Five", "author": "Author C"},
]


If you execute the provided code snippet with this list, it would extract unique authors:

1. It would yield "Author A", "Author B", and "Author C", skipping the book without an author.

### Conclusion

To summarize:
- The code extracts unique authors from a list of books using set comprehension.
- `yield from` efficiently yields each unique author one by one.
- It provides a compact way to both filter and collect necessary data, which is a common task in programming.

### Further Questions and Resources

If you have any questions about how set comprehensions work, generators, or if you want to see more examples relating to this topic, feel free to ask! For further exploration, you might consider looking into:
- Python's official documentation about [Generators](https://docs.python.org/3/tutorial/classes.html#generators).
- Understanding [Comprehensions](https://docs.python.org/3/tutorial/datastructures.html#list-comprehensions) in Python for additional contexts beyond sets.

In [9]:
# Get Llama 3.2 to answer

def explain():
    response = ollama.chat(model=MODEL_LLAMA, messages=tutor_messages)
    return response['message']['content']

def display_explanation():
    explanation = explain()
    display(Markdown(explanation))

In [10]:
display_explanation()

Let's break down this code step by step.

**Overview**

This code is written in Python and uses a feature called "yield from" (introduced in Python 3.3). The main goal of this code is to iterate over the authors of books in a list, without loading the entire list into memory at once.

**Let's examine each part:**

1. `yield from {book.get("author") for book in books if book.get("author")}`

   This expression consists of three parts:

    *   `{book.get("author") for book in books if book.get("author")}`: This is a generator expression, similar to a list comprehension but it generates values on-the-fly rather than storing them all at once. It iterates over the `books` list, and only includes books where the author key exists.
    *   `yield from`: This keyword indicates that this function should yield control back to the caller for each item produced by the inner generator expression (in this case, book authors).
    *   `{...}`: This is an empty dictionary literal. However, in Python 3.5 and later versions, dictionaries can be used as a container for generators expressions.

**How it works**

Here's how we break down the process:

1.  The inner generator expression `book.get("author") for book in books if book.get("author")` generates authors from each book.
2.  It skips books where the author key does not exist (i.e., for which the condition `if book.get("author")` is False).
3.  Since it's using a generator expression, this code doesn't create an entire list of authors before proceeding. Instead, it generates them one by one as needed.
4.  The outer function uses `yield from` to delegate to each item produced by the inner generator expression.

**Key Benefits**

By using this technique:

*   Memory usage is minimized because we don't need to store all the book authors at once in memory.
*   Iterating over a large dataset that you can't fit into memory (like an entire library of books with many authors) becomes more feasible.

Here's a simple example where this code will be useful:
```python
books = [
    {"title": "Book 1", "author": "Author A"},
    {"title": "Book 2", "author": "Author B"},
    # Ignore book with missing author key.
    {"title": "Book 3"},
]

for author in yield_from({book.get("author") for book in books if book.get("author")}):
    print(author)
```
When you run this, it will print the authors of the books without loading all of them into memory at once.

**Summary**

This code is used to generate book authors from a list of books while minimizing memory usage. It's particularly useful for handling large datasets that can't be loaded entirely into memory.